In [69]:
import pandas as pd
import numpy as np
import glob
import os

import warnings
warnings.filterwarnings('ignore')

folder = r"C:\Climate-Data\stjohns\Historical Obs"

In [70]:
csv_files = glob.glob(os.path.join(folder, "*.csv"))
df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
df['LOCAL_DATE'] = pd.to_datetime(df['LOCAL_DATE'])
df

,x,y,STATION_NAME,CLIMATE_IDENTIFIER,ID,LOCAL_DATE,PROVINCE_CODE,LOCAL_YEAR,LOCAL_MONTH,LOCAL_DAY,...,SPEED_MAX_GUST,SPEED_MAX_GUST_FLAG,COOLING_DEGREE_DAYS,COOLING_DEGREE_DAYS_FLAG,HEATING_DEGREE_DAYS,HEATING_DEGREE_DAYS_FLAG,MIN_REL_HUMIDITY,MIN_REL_HUMIDITY_FLAG,MAX_REL_HUMIDITY,MAX_REL_HUMIDITY_FLAG
0,-52.752500,47.618611,ST. JOHN'S INTL A,8403505,8403505.2012.3.20,2012-03-20,NL,2012,3,20,...,70.0,NaN,0.0,NaN,19.6,NaN,NaN,NaN,NaN,NaN
1,-52.752500,47.618611,ST. JOHN'S INTL A,8403505,8403505.2012.3.21,2012-03-21,NL,2012,3,21,...,54.0,NaN,0.0,NaN,23.9,NaN,NaN,NaN,NaN,NaN
2,-52.752500,47.618611,ST. JOHN'S INTL A,8403505,8403505.2012.3.22,2012-03-22,NL,2012,3,22,...,43.0,NaN,0.0,NaN,19.2,NaN,NaN,NaN,NaN,NaN
3,-52.752500,47.618611,ST. JOHN'S INTL A,8403505,8403505.2012.3.23,2012-03-23,NL,2012,3,23,...,70.0,NaN,0.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN
4,-52.752500,47.618611,ST. JOHN'S INTL A,8403505,8403505.2012.3.24,2012-03-24,NL,2012,3,24,...,63.0,NaN,0.0,NaN,22.3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30542,-52.742778,47.622222,ST JOHN'S A,8403506,8403506.2012.3.15,2012-03-15,NL,2012,3,15,...,44.0,E,0.0,NaN,23.6,NaN,NaN,M,NaN,M
30543,-52.742778,47.622222,ST JOHN'S A,8403506,8403506.2012.3.16,2012-03-16,NL,2012,3,16,...,44.0,E,0.0,NaN,19.5,NaN,NaN,M,NaN,M
30544,-52.742778,47.622222,ST JOHN'S A,8403506,8403506.2012.3.17,2012-03-17,NL,2012,3,17,...,39.0,E,0.0,NaN,17.1,NaN,NaN,M,NaN,M
30545,-52.742778,47.622222,ST JOHN'S A,8403506,8403506.2012.3.18,2012-03-18,NL,2012,3,18,...,72.0,E,0.0,NaN,17.0,NaN,NaN,M,NaN,M


In [71]:
df['LOCAL_DATE'].duplicated().sum()

np.int64(0)

In [72]:
ranges = [1991,2020]
df = df[(df['LOCAL_YEAR'] >= ranges[0]) & (df['LOCAL_YEAR'] <= ranges[1])]
winter_df = df[df['LOCAL_DATE'].dt.month.isin([12, 1, 2])]

In [73]:
# daily average temperature annual average, 30 year min, average, max, in degC
dat = df.groupby(df['LOCAL_DATE'].dt.year)['MEAN_TEMPERATURE'].mean()
[dat.min(), dat.mean(), dat.max()]

[np.float64(3.6237704918032785),
 np.float64(5.338828689557558),
 np.float64(6.980487804878049)]

In [74]:
# cold days annual count, 30 year min, average, max, in daycount
df['Cold_Day'] = np.where(df['MIN_TEMPERATURE'] <= -15.0, 1, 0)
cd = df.groupby(df['LOCAL_DATE'].dt.year)['Cold_Day'].sum()
[cd.min(), cd.mean(), cd.max()]

[np.int64(0), np.float64(5.8), np.int64(25)]

In [75]:
# cold waves annual count, 30 year min, average, max, in spellcount
df['Cold_spell'] = df['Cold_Day'].rolling(window=3).sum() == 3
cw = df.groupby(df['LOCAL_DATE'].dt.year)['Cold_spell'].sum()
[cw.min(), cw.mean(), cw.max()]

[np.int64(0), np.float64(1.0666666666666667), np.int64(8)]

In [76]:
# heat waves annual count, 30 year min, average, max, in spellcount
df['Hot_Day'] = np.where(df['MAX_TEMPERATURE'] >= 30.0, 1, 0)
df['Hot_spell'] = df['Hot_Day'].rolling(window=3).sum() == 3
hw = df.groupby(df['LOCAL_DATE'].dt.year)['Hot_spell'].sum()
[hw.min(), hw.mean(), hw.max()]

[np.int64(0), np.float64(0.0), np.int64(0)]

In [77]:
# annual freeze thaw cycles, in daycount
df["Daily Delta"] = df['MAX_TEMPERATURE'] - df['MIN_TEMPERATURE']
df["Daily Product"] = df['MAX_TEMPERATURE'] * df['MIN_TEMPERATURE']
df['Cross_Zero'] = np.where(df['Daily Product'] < 0, 1, 0)
ftc = df.groupby(df['LOCAL_DATE'].dt.year)['Cross_Zero'].sum()
[ftc.min(), ftc.mean(), ftc.max()]

[np.int64(74), np.float64(98.43333333333334), np.int64(118)]

In [78]:
# winter freeze thaw cycles, in daycount
winter_df["Daily Delta"] = winter_df['MAX_TEMPERATURE'] - winter_df['MIN_TEMPERATURE']
winter_df["Daily Product"] = winter_df['MAX_TEMPERATURE'] * winter_df['MIN_TEMPERATURE']
winter_df['Cross_Zero'] = np.where(winter_df['Daily Product'] < 0, 1, 0)
wftc = winter_df.groupby(winter_df['LOCAL_DATE'].dt.year)['Cross_Zero'].sum()
[wftc.min(), wftc.mean(), wftc.max()]

[np.int64(21), np.float64(36.2), np.int64(48)]

In [79]:
# diurnal temperature variation, in daycount
df["Daily Delta"] = df['MAX_TEMPERATURE'] - df['MIN_TEMPERATURE']
df['15C_Variation'] = np.where(df['Daily Delta'] >= 15.0, 1, 0)
dtv = df.groupby(df['LOCAL_DATE'].dt.year)['15C_Variation'].sum()
[dtv.min(), dtv.mean(), dtv.max()]

[np.int64(1), np.float64(14.233333333333333), np.int64(32)]

In [80]:
# heating degree days, in annual count degreedays
hdd = df.groupby(df['LOCAL_DATE'].dt.year)['HEATING_DEGREE_DAYS'].sum()
[hdd.min(), hdd.mean(), hdd.max()]

[np.float64(3676.0), np.float64(4633.63), np.float64(5276.5)]

In [81]:
# cooling degree days, in annual count degreedays
cdd = df.groupby(df['LOCAL_DATE'].dt.year)['COOLING_DEGREE_DAYS'].sum()
[cdd.min(), cdd.mean(), cdd.max()]

[np.float64(14.8), np.float64(45.06666666666667), np.float64(89.4)]

In [82]:
# humidex, in daycount > 35

In [ ]:
# heavy rain
# historical values, a 100 year return
returnrate = 4.6 #mm/h
hourcount = 24 #h
threshold = returnrate * hourcount #mm/day
df['Heavy Rain Day'] = np.where(df['TOTAL_RAIN'] >= threshold, 1, 0)
hr = df.groupby(df['LOCAL_DATE'].dt.year)['Heavy Rain Day'].sum()
["threshold: ", threshold, hr.min(), hr.mean(), hr.max()]

['threshold: ', 110.39999999999999, np.int64(0), np.float64(0.0), np.int64(0)]

In [ ]:
# ice accretion

In [85]:
# rain on snow per year, in mm
rosmm = df[(df['TOTAL_RAIN'] > 0) & (df['SNOW_ON_GROUND'] > 0.0)]
rosmm2 = rosmm.groupby(rosmm['LOCAL_DATE'].dt.year)['TOTAL_RAIN'].sum()
[rosmm2.min(), rosmm2.mean(), rosmm2.max()]


[np.float64(139.2), np.float64(241.89), np.float64(446.4)]

In [95]:
# rain on snow per year, in daycount
rosmm = df[(df['TOTAL_RAIN'] > 0) & (df['SNOW_ON_GROUND'] > 0.0)]
rosmm['Rain on Snow'] = 1 #literally just adding a 1 since i did the filtering beforehand
rosmm3 = rosmm.groupby(rosmm['LOCAL_DATE'].dt.year)['Rain on Snow'].sum()
[rosmm3.min(), rosmm3.mean(), rosmm3.max()]

[np.int64(21), np.float64(35.833333333333336), np.int64(53)]

In [87]:
# annual precipitation, in mm/year
ap = df.groupby(df['LOCAL_DATE'].dt.year)['TOTAL_PRECIPITATION'].sum()
[ap.min(), ap.mean(), ap.max()]

[np.float64(1216.3), np.float64(1512.3366666666666), np.float64(1905.6)]

In [88]:
# winter preciiptation, in mm/year
wp = winter_df.groupby(winter_df['LOCAL_DATE'].dt.year)['TOTAL_PRECIPITATION'].sum()
[wp.min(), wp.mean(), wp.max()]

[np.float64(273.4), np.float64(434.60333333333335), np.float64(584.2)]

In [89]:
# annual snowfall, in cm/year
ans = df.groupby(df['LOCAL_DATE'].dt.year)['TOTAL_SNOW'].sum()
ans = ans/10
[ans.min(), ans.mean(), ans.max()]

[np.float64(17.580000000000002),
 np.float64(35.15066666666667),
 np.float64(51.18)]

wildfires methodology
- created csv of sample point in latlon
- exported to utm 22n, reimported as new layer
- processing toolbox > 50 km buffer circle
- imported list of fire locations from cfwis
- vector > research > files in selection, selected points contained in the buffer
- opened attributes, ctrl c, ctrl v into new csv file

In [ ]:
# wildfires, all fires
firehist = pd.read_csv(r"C:\Users\CAMG038492\OneDrive - WSP O365\Documents\Climate Data\John Cabot GIS\Copy of fires within 50 km of st johns.csv")
firehist = firehist[(firehist['YEAR'] >= ranges[0]) & (firehist['YEAR'] <= ranges[1])]
firehist['YEAR'] = pd.to_datetime(firehist['YEAR'], format = "%Y")
firehist["count"] = 1
wf = firehist.groupby(firehist['YEAR'].dt.year)['count'].sum()
[wf.min(), wf.mean(), wf.max()]



[np.int64(8), np.float64(32.77777777777778), np.int64(84)]

In [91]:
# high winds > 63, in daycount
df['Wind gt 63'] = np.where(df['SPEED_MAX_GUST'] >= 63.0, 1, 0)
hwa = df.groupby(df['LOCAL_DATE'].dt.year)['Wind gt 63'].sum()
[hwa.min(), hwa.mean(), hwa.max()]

[np.int64(46), np.float64(93.16666666666667), np.int64(163)]

In [92]:
# high winds > 90, in daycount
df['Wind gt 90'] = np.where(df['SPEED_MAX_GUST'] >= 90.0, 1, 0)
hwb = df.groupby(df['LOCAL_DATE'].dt.year)['Wind gt 90'].sum()
[hwb.min(), hwb.mean(), hwb.max()]

[np.int64(1), np.float64(12.133333333333333), np.int64(32)]

In [93]:
# hurricanes / tropical storms